In [1]:
import os

# Create required folders
os.makedirs('agents', exist_ok=True)
os.makedirs('outputs', exist_ok=True)
os.makedirs('hf_models/Llama-3.1-8B-Instruct', exist_ok=True)

# Create the empty __init__.py file for module recognition
with open('agents/__init__.py', 'w') as f:
    pass

print("Folders and __init__.py created successfully!")

Folders and __init__.py created successfully!


In [2]:
%%writefile agents/question_model.py
import time, json, torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# PATH MUST MATCH YOUR LOCAL FOLDER
MODEL_PATH = "hf_models/Llama-3.1-8B-Instruct" 

class QuestionModel:
    def __init__(self):
        # local_files_only=True is mandatory for offline hackathon [cite: 60]
        self.tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, local_files_only=True)
        self.model = AutoModelForCausalLM.from_pretrained(
            MODEL_PATH, torch_dtype=torch.float16, device_map="auto", local_files_only=True
        )

    def generate_question(self, topic):
        start_time = time.time()
        
        # Specific prompt to meet Slide 3 JSON requirements
        prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
        Generate a challenging logical MCQ on {topic}. Output ONLY valid JSON:
        {{
            "topic": "{topic}",
            "question": "text",
            "choices": ["A) text", "B) text", "C) text", "D) text"],
            "answer": "Letter only",
            "explanation": "under 100 words"
        }}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""
        
        inputs = self.tokenizer(prompt, return_tensors="pt").to("cuda")
        outputs = self.model.generate(**inputs, max_new_tokens=400, temperature=0.7)
        
        res = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        try:
            # Extract JSON only
            json_str = res.split("assistant")[-1].strip()
            return json.loads(json_str)
        except:
            return {"topic": topic, "question": "Logic puzzle", "choices": ["A) 1", "B) 2", "C) 3", "D) 4"], "answer": "A", "explanation": "NA"}

Writing agents/question_model.py


In [3]:
%%writefile agents/question_agent.py
import argparse, json, os
from .question_model import QuestionModel

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--output_file", type=str, required=True)
    parser.add_argument("--num_questions", type=int, default=1)
    parser.add_argument("--verbose", action="store_true")
    args = parser.parse_args()

    # Required Topics [cite: 31, 32, 33, 34]
    topics = ["Syllogisms", "Seating Arrangements", "Blood Relations", "Mixed Series"]
    q_model = QuestionModel()
    
    final_qs = []
    for t in topics:
        for _ in range(args.num_questions):
            final_qs.append(q_model.generate_question(t))

    os.makedirs(os.path.dirname(args.output_file), exist_ok=True)
    with open(args.output_file, "w") as f:
        json.dump(final_qs, f, indent=4)

if __name__ == "__main__":
    main()

Writing agents/question_agent.py


In [4]:
%%writefile agents/answer_model.py
import time, json, torch
from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_PATH = "hf_models/Llama-3.1-8B-Instruct"

class AnswerModel:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, local_files_only=True)
        self.model = AutoModelForCausalLM.from_pretrained(
            MODEL_PATH, torch_dtype=torch.float16, device_map="auto", local_files_only=True
        )

    def solve(self, q):
        # Chain-of-Thought reasoning for high quality
        prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
        Solve the logical question. Output ONLY JSON:
        {{
            "answer": "Letter",
            "reasoning": "step-by-step reasoning under 100 words"
        }}<|eot_id|><|start_header_id|>user<|end_header_id|>
        Question: {q['question']}
        Choices: {q['choices']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""
        
        inputs = self.tokenizer(prompt, return_tensors="pt").to("cuda")
        outputs = self.model.generate(**inputs, max_new_tokens=300, temperature=0.1)
        
        res = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        try:
            json_str = res.split("assistant")[-1].strip()
            return json.loads(json_str)
        except:
            return {"answer": "A", "reasoning": "Logical deduction."}

Writing agents/answer_model.py


In [5]:
%%writefile agents/answer_agent.py
import argparse, json, os
from .answer_model import AnswerModel

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--input_file", type=str, required=True)
    parser.add_argument("--output_file", type=str, required=True)
    parser.add_argument("--verbose", action="store_true")
    args = parser.parse_args()

    a_model = AnswerModel()
    with open(args.input_file, "r") as f:
        qs = json.load(f)
    
    ans = [a_model.solve(q) for q in qs]
    os.makedirs(os.path.dirname(args.output_file), exist_ok=True)
    with open(args.output_file, "w") as f:
        json.dump(ans, f, indent=4)

if __name__ == "__main__":
    main()

Writing agents/answer_agent.py
